In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('day_04_input.txt', 'r') as inputfile:
    data = inputfile.readlines()

data2 = ['datim,action\n']
for line in data:
    zeile = line.replace('[15', '20').replace(']', ':00.0,')
    zeile = zeile.replace(' Guard #', '').replace(' begins shift', '')
    zeile = zeile.replace(' falls asleep', '-1').replace(' wakes up', '+1')
    zeile = zeile.replace(' ', 'T')
    data2.append(zeile)

with open('day4test.csv', 'w') as outfile:
    for line in data2:
        outfile.write(line)

df = pd.read_csv('day4test.csv')
df['datum'] = pd.to_datetime(df.datim)
df = df.drop('datim', axis=1)
df.sort_values('datum', inplace=True)
df.reset_index(drop=True, inplace=True)

df['bot'] = df.action.copy(deep=True)
df.loc[df.bot == -1, 'bot'] = np.nan
df.loc[df.bot == 1, 'bot'] = np.nan
df.fillna(method='ffill', inplace=True)
df.bot = df.bot.astype('int64')

In [3]:
schedule = dict()
for row, columns in df.iterrows():
    if columns['action'] == columns['bot']:
        bot_number = columns.bot
        try:
            _ = schedule[bot_number]['duration']
        except:
            schedule[bot_number] = {'duration': 0, 'minutes': {}}
    elif columns['action'] == -1:
        asleep = columns.datum
    elif columns['action'] == 1:
        awake = columns.datum
        duration = awake.minute - asleep.minute
        schedule[bot_number]['duration'] += duration
        for minute in range(asleep.minute, awake.minute):
            try:
                schedule[bot_number]['minutes'][minute] += 1
            except:
                schedule[bot_number]['minutes'][minute] = 1

In [4]:
max_bot, max_duration = 0, 0
for bot_number in schedule:
    if schedule[bot_number]['duration'] > max_duration:
        max_duration = schedule[bot_number]['duration']
        max_bot = bot_number
print(f"{max_bot} {max_duration}")

max_minute, max_amount = 0, 0
for minute, amount in schedule[max_bot]['minutes'].items():
    if amount > max_amount:
        max_amount = amount
        max_minute = minute
print(max_minute, max_amount)
print(f"{max_bot * max_minute}")

3203 457
44 13
140932


In [5]:
max_bot, max_minute, max_amount = 0, 0, 0
for bot_number in schedule:
    for minute, amount in schedule[bot_number]['minutes'].items():
        if amount > max_amount:
            max_amount = amount
            max_minute = minute
            max_bot = bot_number
print(max_bot, max_minute, max_amount)
print(f"{max_bot * max_minute}")

1601 32 16
51232
